In [101]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# from scipy import stats
from IPython.display import display

def export(df: pd.DataFrame, path_no_dot: str, index=True):
    df.to_csv(f'exports/{path_no_dot}.csv', index=index)
    df.to_excel(f'exports/{path_no_dot}.xlsx', index=index)

In [102]:
# data initialization

# needs ppt_analysis.ipynb to be ran first
players_table = pd.read_csv('exports/players_table.csv')
matches_table = pd.read_csv('exports/matches_table.csv').sort_values(['event', 'match_number']).reset_index(drop=True)
matches_table.drop('Unnamed: 0', axis=1, inplace=True)

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(matches_table)

,event,match_number,fe1,fe2,t1p1,t1p2,t2p1,t2p2,score1,score2,seed1,seed2,tpc,D_seeds,adjD_seeds,seed1_favor,seed2_favor,D_scores,adjD_scores,upset,abs_upset
0,2021_2s,1,False,False,Rose Roché,Aaron Carter,Brian Tafazoli,Evan Sooklal,6,3,1,8,6,-7,-0.875000,93.750000,6.250000,-3,-0.500000,0.500000,0.500000
1,2021_2s,2,False,False,Coby Lovelace,Jason Jackson,Jack Massingill,Will Simpson,6,3,4,5,6,-1,-0.125000,56.250000,43.750000,-3,-0.500000,-0.357143,0.357143
2,2021_2s,3,False,False,Kevin Cooper,Roman Ramirez,Ann Hall,Anna Brown,6,3,2,7,6,-5,-0.625000,81.250000,18.750000,-3,-0.500000,0.214286,0.214286
3,2021_2s,4,False,False,Rohan Chowla,Levin Lee,Paul Bartenfeld,Leah Baetcke,6,0,3,6,6,-3,-0.375000,68.750000,31.250000,-6,-1.000000,-0.571429,0.571429
4,2021_2s,5,True,True,Brian Tafazoli,Evan Sooklal,Jack Massingill,Will Simpson,4,6,8,5,6,3,0.375000,31.250000,68.750000,2,0.333333,-0.095238,0.095238
5,2021_2s,6,True,True,Ann Hall,Anna Brown,Paul Bartenfeld,Leah Baetcke,6,4,7,6,6,1,0.125000,43.750000,56.250000,-2,-0.333333,-0.476190,0.476190
6,2021_2s,7,False,False,Rose Roché,Aaron Carter,Coby Lovelace,Jason Jackson,5,6,1,4,6,-3,-0.375000,68.750000,31.250000,1,0.166667,0.595238,0.595238
7,2021_2s,8,False,False,Kevin Cooper,Roman Ramirez,Rohan Chowla,Levin Lee,5,6,2,3,6,-1,-0.125000,56.250000,43.750000,1,0.166667,0.309524,0.309524
8,2021_2s,9,True,True,Kevin Cooper,Roman Ramirez,Ann Hall,Anna Brown,6,5,2,7,6,-5,-0.625000,81.250000,18.750000,-1,-0.166667,0.547619,0.547619
9,2021_2s,10,True,True,Rose Roché,Aaron Carter,Jack Massingill,Will Simpson,5,6,1,5,6,-4,-0.500000,75.000000,25.000000,1,0.166667,0.738095,0.738095


In [103]:
def expected_score(ratingA, ratingB): # expected difference in cups hit
    # sigmoid vertical bounds
    yrange = 12
    ymin = 6
    # game variables
    expected_cups = 2
    normal_elo_difference = 400

    # score is the expected difference in cups scored,
        # if this function spits out -5, then players of those two ratings should end in a difference of 5 cups
    # this function is a sigmoid function from -6 to 6
    # this function says an elo difference of "normal_elo_difference" should result in absolute difference in "expected_cups"
        # this works: an elo difference of 400 should result in a score difference of 2
    return (yrange / (1 + np.power(expected_cups, (ratingB - ratingA) / normal_elo_difference))) - ymin

def rating_change(score, expected_score):
    # the k-factor: determines how strongly a result affects the rating change
    # usually between 10 and 40, but with few games, we want to change it frequently
    K = 12
    return K * (score - expected_score)

# ELO INITIALIZATION
starting_elo = 1200.0
elo = dict()
elo_time = np.zeros([len(players_table['player']), len(matches_table) + 1])
d_elo_time = np.zeros([len(players_table['player']), len(matches_table) + 1])
for player in players_table['player']:
    elo.update({player: starting_elo})
    elo_time[players_table['player'][players_table['player'] == player].index[0], 0] = starting_elo

# ALGORITHM
for (i, row) in matches_table.iterrows():
    elos_t1 = list()
    elos_t2 = list()

    elos_t1.append(elo[row.t1p1])
    elos_t2.append(elo[row.t2p1])

    # doubles
    if (type(row.t1p2) == str) and (type(row.t2p2) == str):
        elos_t1.append(elo[row.t1p2])
        elos_t2.append(elo[row.t2p2])

    elo_t1 = np.mean(elos_t1)
    elo_t2 = np.mean(elos_t2)

    win_prob_t1 = expected_score(elo_t1, elo_t2)
    win_prob_t2 = expected_score(elo_t2, elo_t1)

    rating_change_t1p1 = rating_change(row.score1 - row.score2, win_prob_t1)
    rating_change_t2p1 = rating_change(row.score2 - row.score1, win_prob_t2)

    elo[row.t1p1] += rating_change_t1p1
    elo[row.t2p1] += rating_change_t2p1
    
    d_elo_time[players_table['player'][players_table['player'] == row.t1p1].index[0], i + 1] = rating_change_t1p1
    d_elo_time[players_table['player'][players_table['player'] == row.t2p1].index[0], i + 1] = rating_change_t2p1

    elo_time[players_table['player'][players_table['player'] == row.t1p1].index[0], i + 1] = elo[row.t1p1]
    elo_time[players_table['player'][players_table['player'] == row.t2p1].index[0], i + 1] = elo[row.t2p1]

    # doubles
    if (type(row.t1p2) == str) and (type(row.t2p2) == str):

        rating_change_t1p2 = rating_change(row.score1 - row.score2, win_prob_t1)
        rating_change_t2p2 = rating_change(row.score2 - row.score1, win_prob_t2)

        elo[row.t1p2] += rating_change_t1p2
        elo[row.t2p2] += rating_change_t2p2

        d_elo_time[players_table['player'][players_table['player'] == row.t1p2].index[0], i + 1] = rating_change_t1p2
        d_elo_time[players_table['player'][players_table['player'] == row.t2p2].index[0], i + 1] = rating_change_t2p2
        
        elo_time[players_table['player'][players_table['player'] == row.t1p2].index[0], i + 1] = elo[row.t1p2]
        elo_time[players_table['player'][players_table['player'] == row.t2p2].index[0], i + 1] = elo[row.t2p2]

players_table['current_elo'] = players_table['player'].map(elo)

player_elo_table = players_table[['player', 'current_elo']].sort_values('current_elo', ascending=False).reset_index(drop=True)

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(player_elo_table)

export(player_elo_table, 'player_elo_table', False)

,player,current_elo
0,Levin Lee,1391.624867
1,Aaron Carter,1385.287081
2,Roman Ramirez,1376.249823
3,Rohan Chowla,1277.890315
4,Kevin Cooper,1271.813140
5,Will Simpson,1244.496624
6,Nathan Snow,1234.327352
7,Gabe Silverstein,1233.467836
8,Coby Lovelace,1230.531071
9,Jack Massingill,1228.341893


In [104]:
print("A much higher ELO, B barely wins: A's change is", rating_change(5 - 6, expected_score(1400, 1000)))
print("A much higher ELO, A barely wins: A's change is", rating_change(6 - 5, expected_score(1400, 1000)))
print("A much higher ELO, B hugely wins: A's change is", rating_change(1 - 6, expected_score(1400, 1000)))
print("A much higher ELO, A hugely wins: A's change is", rating_change(6 - 1, expected_score(1400, 1000)))
print()
print("A closely higher ELO, B barely wins: A's change is", rating_change(5 - 6, expected_score(1250, 1200)))
print("A closely higher ELO, A barely wins: A's change is", rating_change(6 - 5, expected_score(1250, 1200)))
print("A closely higher ELO, B hugely wins: A's change is", rating_change(1 - 6, expected_score(1250, 1200)))
print("A closely higher ELO, A hugely wins: A's change is", rating_change(6 - 1, expected_score(1250, 1200)))
print()
print("A closely lower ELO, B barely wins: A's change is", rating_change(5 - 6, expected_score(1200, 1250)))
print("A closely lower ELO, A barely wins: A's change is", rating_change(6 - 5, expected_score(1200, 1250)))
print("A closely lower ELO, B hugely wins: A's change is", rating_change(1 - 6, expected_score(1200, 1250)))
print("A closely lower ELO, A hugely wins: A's change is", rating_change(6 - 1, expected_score(1200, 1250)))
print()
print("A much lower ELO, B barely wins: A's change is", rating_change(5 - 6, expected_score(1000, 1400)))
print("A much lower ELO, A barely wins: A's change is", rating_change(6 - 5, expected_score(1000, 1400)))
print("A much lower ELO, B hugely wins: A's change is", rating_change(1 - 6, expected_score(1000, 1400)))
print("A much lower ELO, A hugely wins: A's change is", rating_change(6 - 1, expected_score(1000, 1400)))
print()

A much higher ELO, B barely wins: A's change is -36.0
A much higher ELO, A barely wins: A's change is -12.0
A much higher ELO, B hugely wins: A's change is -84.0
A much higher ELO, A hugely wins: A's change is 36.0

A closely higher ELO, B barely wins: A's change is -15.117212459956022
A closely higher ELO, A barely wins: A's change is 8.882787540043978
A closely higher ELO, B hugely wins: A's change is -63.11721245995602
A closely higher ELO, A hugely wins: A's change is 56.88278754004398

A closely lower ELO, B barely wins: A's change is -8.882787540043967
A closely lower ELO, A barely wins: A's change is 15.117212459956033
A closely lower ELO, B hugely wins: A's change is -56.882787540043964
A closely lower ELO, A hugely wins: A's change is 63.117212459956036

A much lower ELO, B barely wins: A's change is 12.0
A much lower ELO, A barely wins: A's change is 36.0
A much lower ELO, B hugely wins: A's change is -36.0
A much lower ELO, A hugely wins: A's change is 84.0



In [105]:
elo_time_table = pd.concat([players_table.player, pd.DataFrame(elo_time)], axis=1).T
elo_time_table.columns = players_table.player
elo_time_table = elo_time_table.iloc[2:,:]
elo_time_table.replace(0.0, np.nan, inplace=True)

d_elo_time_table = pd.concat([players_table.player, pd.DataFrame(d_elo_time)], axis=1).T
d_elo_time_table.columns = players_table.player
d_elo_time_table = d_elo_time_table.iloc[2:,:]
d_elo_time_table.replace(0.0, np.nan, inplace=True)

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(elo_time_table)

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(d_elo_time_table)

export(elo_time_table, 'elo_time_table', True)

player,Aaron Carter,Ann Hall,Anna Brown,Brian Tafazoli,Carla Betancourt,Cason Duszak,Cassie Deering,Coby Lovelace,Evan Sooklal,Gabe Silverstein,Helen Dunn,Jack Massingill,Jason Jackson,Kevin Cooper,Kristian Banlaoi,Leah Baetcke,Levin Lee,Matthew Rusten,Nathan Snow,Noah Dale,Paul Bartenfeld,Piper Parker,Reagan Fryatt,Rohan Chowla,Roman Ramirez,Rose Roché,Sam Carswell-Tellis,Will Simpson,Yvonne Nguyen
1,1236.000000,NaN,NaN,1164.0,NaN,NaN,NaN,NaN,1164.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1236.000000,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1236.000000,NaN,NaN,NaN,1164.000000,1236.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1164.000000,NaN
3,NaN,1164.000000,1164.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1236.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1236.000000,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1128.000000,1272.000000,NaN,NaN,NaN,1128.000000,NaN,NaN,1272.000000,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,1140.0,NaN,NaN,NaN,NaN,1140.000000,NaN,NaN,1188.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1188.000000,NaN
6,NaN,1185.754931,1185.754931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1106.245069,NaN,NaN,NaN,NaN,1106.245069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1224.000000,NaN,NaN,NaN,NaN,NaN,NaN,1248.000000,NaN,NaN,NaN,NaN,1248.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1224.000000,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1226.245069,NaN,NaN,1281.754931,NaN,NaN,NaN,NaN,NaN,NaN,1281.754931,1226.245069,NaN,NaN,NaN,NaN
9,NaN,1176.279802,1176.279802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1235.720198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1235.720198,NaN,NaN,NaN,NaN
10,1209.754931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1202.245069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1209.754931,NaN,1202.245069,NaN


player,Aaron Carter,Ann Hall,Anna Brown,Brian Tafazoli,Carla Betancourt,Cason Duszak,Cassie Deering,Coby Lovelace,Evan Sooklal,Gabe Silverstein,Helen Dunn,Jack Massingill,Jason Jackson,Kevin Cooper,Kristian Banlaoi,Leah Baetcke,Levin Lee,Matthew Rusten,Nathan Snow,Noah Dale,Paul Bartenfeld,Piper Parker,Reagan Fryatt,Rohan Chowla,Roman Ramirez,Rose Roché,Sam Carswell-Tellis,Will Simpson,Yvonne Nguyen
1,36.000000,NaN,NaN,-36.0,NaN,NaN,NaN,NaN,-36.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.000000,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.000000,NaN,NaN,NaN,-36.000000,36.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-36.000000,NaN
3,NaN,-36.000000,-36.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.000000,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-72.000000,72.000000,NaN,NaN,NaN,-72.000000,NaN,NaN,72.000000,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,-24.0,NaN,NaN,NaN,NaN,-24.000000,NaN,NaN,24.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.000000,NaN
6,NaN,21.754931,21.754931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-21.754931,NaN,NaN,NaN,NaN,-21.754931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-12.000000,NaN,NaN,NaN,NaN,NaN,NaN,12.000000,NaN,NaN,NaN,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-12.000000,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.754931,NaN,NaN,9.754931,NaN,NaN,NaN,NaN,NaN,NaN,9.754931,-9.754931,NaN,NaN,NaN,NaN
9,NaN,-9.475130,-9.475130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.475130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.475130,NaN,NaN,NaN,NaN
10,-14.245069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.245069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-14.245069,NaN,14.245069,NaN


In [106]:
sorted(elo.items(), key=lambda x:x[1], reverse=True)

[('Levin Lee', 1391.624867430988),
 ('Aaron Carter', 1385.287080895593),
 ('Roman Ramirez', 1376.2498226627852),
 ('Rohan Chowla', 1277.8903150826286),
 ('Kevin Cooper', 1271.8131397699385),
 ('Will Simpson', 1244.4966236580235),
 ('Nathan Snow', 1234.3273522766856),
 ('Gabe Silverstein', 1233.4678361821286),
 ('Coby Lovelace', 1230.5310714132745),
 ('Jack Massingill', 1228.3418929663865),
 ('Kristian Banlaoi', 1216.037581761179),
 ('Ann Hall', 1210.2255736742936),
 ('Yvonne Nguyen', 1204.8681303937876),
 ('Leah Baetcke', 1203.2293796154731),
 ('Piper Parker', 1194.7488192950173),
 ('Carla Betancourt', 1192.2582452753336),
 ('Cason Duszak', 1183.6519112266158),
 ('Reagan Fryatt', 1182.605339128595),
 ('Anna Brown', 1176.2798016619008),
 ('Rose Roché', 1145.1104962354598),
 ('Helen Dunn', 1143.9885906862953),
 ('Noah Dale', 1143.9885906862953),
 ('Jason Jackson', 1140.2201152719836),
 ('Brian Tafazoli', 1140.0),
 ('Matthew Rusten', 1139.327236294392),
 ('Cassie Deering', 1113.9165304897

In [114]:
import plotly.graph_objects as go

fig = go.Figure()

ranking = 1
for (player,current_rating) in sorted(elo.items(), key=lambda x:x[1], reverse=True):
    fig.add_trace(go.Scatter(
        x=elo_time_table.index,
        y=elo_time_table[player],
        name=f'#{ranking} ({current_rating:.0f}) {player}',
        mode='lines+markers',
        line_shape='hv',
        connectgaps=True,
        text=d_elo_time_table[player],
    ))
    ranking += 1

# generalized
vrect_colors = ['green', 'red', 'yellow', 'blue']
seasons = ['2021_2s', '2022_1s', '2022_2s', '2023_1s']
season_labels = ['2021<br>Doubles', '2022<br>Singles', '2022<br>Doubles', '2023<br>Singles']

for (i, season) in enumerate(seasons):
    fig.add_vrect(
        annotation_text=season_labels[i],
        annotation_position="top left",
        x0=matches_table['event'][matches_table['event'] == season].index[0] + 1,
        x1=matches_table['event'][matches_table['event'] == season].index[-1] + 1,
        fillcolor=vrect_colors[i % len(vrect_colors)],
        opacity=0.1,
        line_width=0,
    )


fig.update_layout(
    title='<b>UVA Drumline Pong ELO Rating System</b> <br><i>by Roman Ramirez<i>',
    xaxis_title='<b>Game Number</b>',
    yaxis_title='<b>ELO Rating</b>',
)

team1_list = list()
team2_list = list()
for (i, row) in matches_table.iterrows():
    if type(row.t1p2) is not str:
        team1_list.append(row.t1p1)
    else:
        team1_list.append(row.t1p1 + " and " + row.t1p2)

    if type(row.t2p2) is not str:
        team2_list.append(row.t2p1)
    else:
        team2_list.append(row.t2p1 + " and " + row.t2p2)

customdata = np.stack((team1_list, team2_list, matches_table['score1'], matches_table['score2']), axis=-1)
hovertemplate = (
    '<i>Game %{x}</i> <br>' +
    '<b>%{customdata[0]} vs. %{customdata[1]}</b><br>' +
    '<b>Final Score:</b> %{customdata[2]}-%{customdata[3]}<br>' + 
    '<b>ELO Change:</b> %{text:,.0f}<br>' + 
    '<b>New ELO:</b> %{y:,.0f}<br>'
)

fig.update_traces(
    customdata=customdata,
    hovertemplate=hovertemplate
)

fig.show()
fig.write_html("index.html")

In [108]:

for searcher in ['2021_2s', '2022_1s', '2022_2s', '2023_1s']:
    print(matches_table['event'][matches_table['event'] == searcher].index[0] + 1)
    print(matches_table['event'][matches_table['event'] == searcher].index[-1] + 1)

1
14
15
43
44
60
61
97


In [109]:

with pd.option_context('display.max_rows', 100, 'display.max_columns', None):
    display(matches_table)

,event,match_number,fe1,fe2,t1p1,t1p2,t2p1,t2p2,score1,score2,seed1,seed2,tpc,D_seeds,adjD_seeds,seed1_favor,seed2_favor,D_scores,adjD_scores,upset,abs_upset
0,2021_2s,1,False,False,Rose Roché,Aaron Carter,Brian Tafazoli,Evan Sooklal,6,3,1,8,6,-7,-0.875000,93.750000,6.250000,-3,-0.500000,0.500000,0.500000
1,2021_2s,2,False,False,Coby Lovelace,Jason Jackson,Jack Massingill,Will Simpson,6,3,4,5,6,-1,-0.125000,56.250000,43.750000,-3,-0.500000,-0.357143,0.357143
2,2021_2s,3,False,False,Kevin Cooper,Roman Ramirez,Ann Hall,Anna Brown,6,3,2,7,6,-5,-0.625000,81.250000,18.750000,-3,-0.500000,0.214286,0.214286
3,2021_2s,4,False,False,Rohan Chowla,Levin Lee,Paul Bartenfeld,Leah Baetcke,6,0,3,6,6,-3,-0.375000,68.750000,31.250000,-6,-1.000000,-0.571429,0.571429
4,2021_2s,5,True,True,Brian Tafazoli,Evan Sooklal,Jack Massingill,Will Simpson,4,6,8,5,6,3,0.375000,31.250000,68.750000,2,0.333333,-0.095238,0.095238
5,2021_2s,6,True,True,Ann Hall,Anna Brown,Paul Bartenfeld,Leah Baetcke,6,4,7,6,6,1,0.125000,43.750000,56.250000,-2,-0.333333,-0.476190,0.476190
6,2021_2s,7,False,False,Rose Roché,Aaron Carter,Coby Lovelace,Jason Jackson,5,6,1,4,6,-3,-0.375000,68.750000,31.250000,1,0.166667,0.595238,0.595238
7,2021_2s,8,False,False,Kevin Cooper,Roman Ramirez,Rohan Chowla,Levin Lee,5,6,2,3,6,-1,-0.125000,56.250000,43.750000,1,0.166667,0.309524,0.309524
8,2021_2s,9,True,True,Kevin Cooper,Roman Ramirez,Ann Hall,Anna Brown,6,5,2,7,6,-5,-0.625000,81.250000,18.750000,-1,-0.166667,0.547619,0.547619
9,2021_2s,10,True,True,Rose Roché,Aaron Carter,Jack Massingill,Will Simpson,5,6,1,5,6,-4,-0.500000,75.000000,25.000000,1,0.166667,0.738095,0.738095
